In [47]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn as nn
from torch.autograd import Variable
import random

n_hidden = 128



In [42]:
# Data preprocessing
xtrain = np.load('obj/x_train.npy')
ytrain = np.load('obj/y_train_small.npy')
xval = np.load('obj/x_val.npy')
yval = np.load('obj/y_val.npy')
vocab_dict = np.load('obj/vocab_dict.npy')#word:int

vocab_dict_inv = np.load('obj/vocab_dict_inv.npy')#int:word
vocab_dict = vocab_dict.item()
vocab_dict_inv = vocab_dict_inv.item()
n_letters = 8000
n_categories = 8000
no_of_train_examples = 8640

xtrain = xtrain[0:no_of_train_examples,:]
ytrain = ytrain[0:no_of_train_examples]
print xtrain.shape
print ytrain.shape

(8640, 3)
(8640,)


In [43]:
#Input data preprocessing
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][line[li]] = 1
    return tensor
p = lineToTensor(xtrain[0,:])
print p.shape
#print p

torch.Size([3, 1, 8000])


In [16]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))
    
rnn = RNN(n_letters, n_hidden, n_categories)

In [18]:
input = Variable(lineToTensor(xtrain[0,:]))
hidden = Variable(torch.zeros(1, n_hidden))

output, next_hidden = rnn(input[0], hidden)
print(output)




Variable containing:
-8.9697 -9.0058 -8.9972  ...  -8.9695 -8.9956 -8.9851
[torch.FloatTensor of size 1x8000]



In [19]:
def categoryFromOutput(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    print category_i
    return vocab_dict_inv[category_i], category_i

In [38]:
print(categoryFromOutput(output))

('matches', 639)


In [83]:
def randomTrainingExample():
    idx = random.randint(0,  no_of_train_examples)
    category_tensor = Variable(torch.LongTensor([int(ytrain[idx])]))
    line_tensor = Variable(lineToTensor(xtrain[idx,:]))
    return category_tensor, line_tensor


In [84]:
category_tensor, line_tensor = randomTrainingExample()
print category_tensor

Variable containing:
 6608
[torch.LongTensor of size 1]



In [94]:
criterion = nn.NLLLoss()
learning_rate = 0.005
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.data[0]

In [ ]:
import time
import math

n_iters = 100
print_every = 5000
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        #correct = '✓' if guess == category else '✗ (%s)' % category
        #print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))
        #correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f  / %s ' % (iter, iter / n_iters * 100, timeSince(start), loss, guess))


    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

In [ ]:
category_tensor, line_tensor = randomTrainingExample()
output, loss = train(category_tensor, line_tensor)
current_loss += loss


Variable containing:
-8.9915 -9.0022 -8.9816  ...  -8.9839 -8.9953 -8.9861
[torch.FloatTensor of size 1x8000]

Variable containing:
 6608
[torch.LongTensor of size 1]

